<a href="https://colab.research.google.com/github/CoWoGeo/PUS2022_CWolk/blob/main/ProjectLathamWolk/311_Requests_2020_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

We are trying to find resident complaints that didn't make it all the way to an L&I violation or didn't get categorized as L&I. For example, Streets Department violations probably aren't in L&I data, or a call asking how to report an odor or dumping wouldn't show up in L&I violation records, especially if it never got attached to the junkyard it's about. 311 is the hotline and website for general complaints and info requests in Philly (for matters less urgent than you'd call 911 for), including both questions about how to report something and actual reports. After exploring this data, it seems like there isn't much that's relevant to our project. 

## Process:
After importing the data and exploring it, I removed:

*   Rows without a subject
*   Form auto-replies to emails
*   Rows with no geometry or address information
*   Subjects of "Graffiti Removal" and "Street Light Outage"
*   Complaints/requests related to Parks & Rec properties
*   Subjects about trash and recycling collection. There is a chance that some of these are related to junkyards and not just missed collections or messes left by sanitation trucks, but there isn't enough detail to know, and that would likely be a small portion of the huge number of rows with this subject.

Then I tried various keywords like smoke, junk, trash, auto, fire, dump, etc. to find potential hits. Most of the results for smoke were requesting smoke detectors, and most of the results for auto and car were about abandoned cars (likely on the street).

I then got all of the subjects from the subject line and selected everything that could be junkyard-related. Even after doing that, there were still over 200,000 rows (down from over 600,000). This is from just one year. It wasn't worth the time to go any further.


# Installing Libraries

In [ ]:
!apt install python3-rtree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources
Suggested packages:
  python3-setuptools
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources python3-rtree
0 upgraded, 5 newly installed, 0 to remove and 7 not upgraded.
Need to get 671 kB of archives.
After this operation, 3,948 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 amd64 1.8.5-5 [219 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-c4v5 amd64 1.8.5-5 [51.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-pkg-

In [ ]:
!pip install geopandas
import geopandas as gpd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.9 MB/s 
     |████████████████████████████████| 16.6 MB 132 kB/s 
     |████████████████████████████████| 7.8 MB 48.9 MB/s 


In [ ]:
! pip install shapely
! conda install -c conda-forge fiona

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/bin/bash: conda: command not found


In [ ]:
!pip install cartoframes
import cartoframes as cf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 245 kB 5.1 MB/s 
     |████████████████████████████████| 235 kB 51.3 MB/s 
  Created wheel for carto: filename=carto-1.11.3-py3-none-any.whl size=35088 sha256=a397c702a69e5df490bf8cbc33e482e38f5efc17b51bf695056b68a9f1c3c821
  Stored in directory: /root/.cache/pip/wheels/6b/a3/41/90fa4334cd280f91d17226f36db7a34b12902f7e5b281afb84
  Created wheel for pyrestcli: filename=pyrestcli-0.6.11-py3-none-any.whl size=8496 sha256=996b35cf5cb9f18ec06c4cc7bc1dd894cc099e797921a983c89ec125bfd31e44
  Stored in directory: /root/.cache/pip/wheels/92/fc/98/2e46573b8e1bbe953345578314c765d3ecdc9532450bb90d79
Successfully built carto pyrestcli


In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

# Importing 311 Request Data

Carto was really confusing. Instead of spending more time trying to get it to work, I downloaded the prepackaged 2020 data from Open Data Philly to explore and see if 311 data was worth the effort.

In [ ]:
requests311_2020 = gpd.GeoDataFrame.from_file("https://phl.carto.com/api/v2/sql?filename=public_cases_fc&format=shp&skipfields=cartodb_id&q=SELECT%20*%20FROM%20public_cases_fc%20WHERE%20requested_datetime%20%3E=%20%272020-01-01%27%20AND%20requested_datetime%20%3C%20%272021-01-01%27")

In [ ]:
requests311_2020.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 621924 entries, 0 to 621923
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   objectid    621924 non-null  float64 
 1   service_re  621924 non-null  float64 
 2   subject     621920 non-null  object  
 3   status      621924 non-null  object  
 4   status_not  252397 non-null  object  
 5   service_na  621924 non-null  object  
 6   service_co  560489 non-null  object  
 7   agency_res  621857 non-null  object  
 8   service_no  212412 non-null  object  
 9   requested_  621924 non-null  object  
 10  updated_da  621924 non-null  object  
 11  expected_d  576796 non-null  object  
 12  closed_dat  10593 non-null   object  
 13  address     262042 non-null  object  
 14  zipcode     11177 non-null   object  
 15  media_url   42270 non-null   object  
 16  lat         252973 non-null  float64 
 17  lon         252973 non-null  float64 
 18  geometry    2529

In [ ]:
requests311_2020.describe()

## Cleaning the Data

In [ ]:
# removing requests that have no subject
requests311_2020 = requests311_2020[requests311_2020["subject"].notna()]

In [ ]:
# after some exploration, removing rows that are just auto-reply emails
requests311_2020 = requests311_2020[~requests311_2020["subject"].str.contains("Auto-Reply")]

In [ ]:
requests311_2020.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 537002 entries, 0 to 621923
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   objectid    537002 non-null  float64 
 1   service_re  537002 non-null  float64 
 2   subject     537002 non-null  object  
 3   status      537002 non-null  object  
 4   status_not  252391 non-null  object  
 5   service_na  537002 non-null  object  
 6   service_co  475567 non-null  object  
 7   agency_res  536935 non-null  object  
 8   service_no  212412 non-null  object  
 9   requested_  537002 non-null  object  
 10  updated_da  537002 non-null  object  
 11  expected_d  491874 non-null  object  
 12  closed_dat  10593 non-null   object  
 13  address     262040 non-null  object  
 14  zipcode     11177 non-null   object  
 15  media_url   42270 non-null   object  
 16  lat         252972 non-null  float64 
 17  lon         252972 non-null  float64 
 18  geometry    2529

In [ ]:
# trying to find junkyards! junkyard, scrapyard, etc. didn't find anything
requests311_2020[requests311_2020["subject"].str.contains("smoke")]

,objectid,service_re,subject,status,status_not,service_na,service_co,agency_res,service_no,requested_,updated_da,expected_d,closed_dat,address,zipcode,media_url,lat,lon,geometry
129787,7734178.0,13064857.0,How do I request a free smoke detector?,Closed,Question Answered,Information Request,SR-IR01,Fire Department,None,2020-01-02,2020-01-02,2020-01-02,None,None,None,None,NaN,NaN,None
144874,9528961.0,13781102.0,How do I request a free smoke detector?,Closed,Question Answered,Information Request,SR-IR01,Fire Department,None,2020-11-19,2020-11-19,2020-11-19,None,None,None,None,NaN,NaN,None
145692,7883474.0,13125092.0,How do I request a free smoke detector?,Closed,Question Answered,Information Request,SR-IR01,Fire Department,None,2020-02-04,2020-02-04,2020-02-04,None,None,None,None,NaN,NaN,None
168521,7883510.0,13125106.0,How do I request a free smoke detector?,Closed,Question Answered,Information Request,SR-IR01,Fire Department,None,2020-02-04,2020-02-04,2020-02-04,None,None,None,None,NaN,NaN,None
172183,7972971.0,13177752.0,How do I request a free smoke detector?,Closed,Question Answered,Information Request,SR-IR01,Fire Department,None,2020-02-19,2020-02-19,2020-02-19,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582718,9565197.0,13793274.0,How do I request a free smoke detector?,Closed,Question Answered,Information Request,SR-IR01,Fire Department,None,2020-11-30,2020-11-30,2020-11-30,None,None,None,None,NaN,NaN,None
585062,9653813.0,13830559.0,How do I request a free smoke detector?,Closed,Question Answered,Information Request,SR-IR01,Fire Department,None,2020-12-21,2020-12-21,2020-12-21,None,None,None,None,NaN,NaN,None
587463,9633071.0,13821403.0,How do I request a free smoke detector?,Closed,Question Answered,Information Request,SR-IR01,Fire Department,None,2020-12-15,2020-12-15,2020-12-15,None,None,None,None,NaN,NaN,None
588202,9630327.0,13820093.0,How do I request a free smoke detector?,Closed,Question Answered,Information Request,SR-IR01,Fire Department,None,2020-12-15,2020-12-15,2020-12-15,None,None,None,None,NaN,NaN,None


In [ ]:
# seeing if all the smoke hits are about smoke detectors. they are!
requests311_2020[requests311_2020["subject"].str.contains("How do I request a free smoke detector?")]

,objectid,service_re,subject,status,status_not,service_na,service_co,agency_res,service_no,requested_,updated_da,expected_d,closed_dat,address,zipcode,media_url,lat,lon,geometry
129787,7734178.0,13064857.0,How do I request a free smoke detector?,Closed,Question Answered,Information Request,SR-IR01,Fire Department,None,2020-01-02,2020-01-02,2020-01-02,None,None,None,None,NaN,NaN,None
144874,9528961.0,13781102.0,How do I request a free smoke detector?,Closed,Question Answered,Information Request,SR-IR01,Fire Department,None,2020-11-19,2020-11-19,2020-11-19,None,None,None,None,NaN,NaN,None
145692,7883474.0,13125092.0,How do I request a free smoke detector?,Closed,Question Answered,Information Request,SR-IR01,Fire Department,None,2020-02-04,2020-02-04,2020-02-04,None,None,None,None,NaN,NaN,None
168521,7883510.0,13125106.0,How do I request a free smoke detector?,Closed,Question Answered,Information Request,SR-IR01,Fire Department,None,2020-02-04,2020-02-04,2020-02-04,None,None,None,None,NaN,NaN,None
172183,7972971.0,13177752.0,How do I request a free smoke detector?,Closed,Question Answered,Information Request,SR-IR01,Fire Department,None,2020-02-19,2020-02-19,2020-02-19,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582718,9565197.0,13793274.0,How do I request a free smoke detector?,Closed,Question Answered,Information Request,SR-IR01,Fire Department,None,2020-11-30,2020-11-30,2020-11-30,None,None,None,None,NaN,NaN,None
585062,9653813.0,13830559.0,How do I request a free smoke detector?,Closed,Question Answered,Information Request,SR-IR01,Fire Department,None,2020-12-21,2020-12-21,2020-12-21,None,None,None,None,NaN,NaN,None
587463,9633071.0,13821403.0,How do I request a free smoke detector?,Closed,Question Answered,Information Request,SR-IR01,Fire Department,None,2020-12-15,2020-12-15,2020-12-15,None,None,None,None,NaN,NaN,None
588202,9630327.0,13820093.0,How do I request a free smoke detector?,Closed,Question Answered,Information Request,SR-IR01,Fire Department,None,2020-12-15,2020-12-15,2020-12-15,None,None,None,None,NaN,NaN,None


In [ ]:
# removing rows with no geometry
requests311_2020 = requests311_2020[requests311_2020["geometry"].notna()]

In [ ]:
#removing those. a drop in the bucket...
requests311_2020 = requests311_2020[~requests311_2020["subject"].str.contains("How do I request a free smoke detector?")]

In [ ]:
# removing parks and rec maintenance
requests311_2020 = requests311_2020[~requests311_2020["subject"].str.contains("Parks and Rec Safety and Maintenance")]

In [ ]:
requests311_2020["service_na"].value_counts()

Rubbish/Recyclable Material Collection    47692
Illegal Dumping                           26250
Maintenance Complaint                     26210
Abandoned Vehicle                         24539
Information Request                       18504
                                          ...  
Newsstand Outdoor Cafe                        4
Vacant Lot Clean-Up                           4
Fire_Safety_Complaint                         2
No Heat Residential                           2
Right-of-Way                                  1
Name: service_na, Length: 62, dtype: int64

I'm just iterating over the service counts and removing large counts that seem irrelevant. I think I could have halved the size of this by removing Info Requests or no geometry to start with. Originally, I cut all Information Request service_nas, but I'm going to try leaving them in.

In [ ]:
requests311_2020 = requests311_2020[~requests311_2020["service_na"].str.contains("Graffiti Removal")]
requests311_2020 = requests311_2020[~requests311_2020["service_na"].str.contains("Street Light Outage")]

In [ ]:
requests311_2020 = requests311_2020[~requests311_2020["service_na"].str.contains("Rubbish/Recyclable Material Collection")]

In [ ]:
# what subjects are present?
requests311_2020["subject"].nunique()

836

In [ ]:
requests311_2020.head(10)

,objectid,service_re,subject,status,status_not,service_na,service_co,agency_res,service_no,requested_,updated_da,expected_d,closed_dat,address,zipcode,media_url,lat,lon,geometry
0,13856824.0,13697322.0,Smoke Detector,Closed,The customer cancelled the request.,Smoke Detector,SR-FD02,Fire Department,60 Business Days,2020-09-29,2022-05-17,2021-01-03,None,241 W CALVERT ST,None,None,40.039849,-75.125438,POINT (-75.12544 40.03985)
1,12165628.0,13315073.0,Illegal Dumping,Closed,None,Illegal Dumping,SR-ST02,Streets Department,5 Business Days,2020-04-25,2022-03-12,2020-05-03,None,3100 W THOMPSON ST,None,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.976803,-75.187074,POINT (-75.18707 39.97680)
2,12165629.0,13315109.0,Illegal Dumping,Closed,None,Illegal Dumping,SR-ST02,Streets Department,5 Business Days,2020-04-25,2022-03-12,2020-05-03,None,2401 S EDGEWOOD ST,None,None,39.927871,-75.225099,POINT (-75.22510 39.92787)
3,12165603.0,13312660.0,Illegal Dumping,Closed,None,Illegal Dumping,SR-ST02,Streets Department,5 Business Days,2020-04-23,2022-03-12,2020-04-30,None,325 E ALLEGHENY AVE,None,None,39.997971,-75.124529,POINT (-75.12453 39.99797)
4,12165604.0,13312758.0,Illegal Dumping,Closed,None,Illegal Dumping,SR-ST02,Streets Department,5 Business Days,2020-04-23,2022-03-12,2020-04-30,None,2726 MANTON ST,None,None,39.938627,-75.189384,POINT (-75.18938 39.93863)
5,12165648.0,13486115.0,Illegal Dumping,Closed,None,Illegal Dumping,SR-ST02,Streets Department,5 Business Days,2020-07-13,2022-03-12,2020-07-19,None,708 EMILY ST,None,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.923714,-75.158356,POINT (-75.15836 39.92371)
6,12165649.0,13486196.0,Illegal Dumping,Closed,None,Illegal Dumping,SR-ST02,Streets Department,5 Business Days,2020-07-13,2022-03-12,2020-07-19,None,3028 MARTHA ST,None,None,39.989465,-75.113036,POINT (-75.11304 39.98946)
7,12165601.0,13312482.0,Illegal Dumping,Closed,None,Illegal Dumping,SR-ST02,Streets Department,5 Business Days,2020-04-23,2022-03-12,2020-04-30,None,2401 SNYDER AVE,None,None,39.926567,-75.186514,POINT (-75.18651 39.92657)
8,12165602.0,13312560.0,Illegal Dumping,Closed,None,Illegal Dumping,SR-ST02,Streets Department,5 Business Days,2020-04-23,2022-03-12,2020-04-30,None,4900 N 3RD ST,None,None,40.025469,-75.129984,POINT (-75.12998 40.02547)
9,12165630.0,13315198.0,Illegal Dumping,Closed,None,Illegal Dumping,SR-ST02,Streets Department,5 Business Days,2020-04-25,2022-03-12,2020-05-03,None,E CHELTENHAM AVE & HARBISON AVE,None,None,40.017513,-75.065115,POINT (-75.06511 40.01751)


In [ ]:
requests311_2020["subject"].unique()

array(['Smoke Detector', 'Illegal Dumping', 'Complaint (Streets)',
       'Maintenance Complaint', 'Construction Complaints',
       'Right of Way Unit', 'Homeless Encampment Request',
       'Other (Streets)', 'Fire Safety Complaint', 'Illegal dumping',
       'Inlet Cleaning', 'LI Escalation',
       'How do I report businesses operating during mandated COVID-19 closure?',
       'How do I report individuals not complying with the Stay At Home Order in Philadelphia?',
       'Abandoned Automobile', 'Rubbish/Recyclable Material Collection',
       'Abandoned Vehicle', 'License Complaint',
       'Alleys, Driveways & Retaining Walls',
       'Dangerous Building Complaint', 'Street Light Outage',
       'Dangerous Sidewalk', 'Alley Light Outage', 'Street Defect',
       'Complaints Against Fire or EMS', 'Complaints against Fire or EMS',
       'Complaint Against Fire or EMS', 'Maintenance Complaints',
       'Vacant House or Commercial',
       'What numbers do I call to contact PECO an

In [ ]:
# building a list of subjects I want to keep
subjects = ['Illegal Dumping', 'Complaint (Streets)', 'Maintenance Complaint',
            'Right of Way Unit', 'Other (Streets)', 'Fire Safety Complaint', 
            'Illegal dumping', 'Inlet Cleaning', 'LI Escalation', 'LI Escalation',
            'Abandoned Automobile', 'Abandoned Vehicle', 'License Complaint', 
            'Alleys, Driveways & Retaining Walls', 
            'Dangerous Building Complaint', 'Dangerous Sidewalk', 
            'Street Defect', 'Complaints Against Fire or EMS', 
            'Complaints against Fire or EMS', 'Complaint Against Fire or EMS', 
            'Maintenance Complaints', 'Maintenance Residential or Commercial', 
            'Building Force', 'Sanitation / Dumpster Violation', 
            'Other Dangerous', 'Building Dangerous', 'Vacant Lot Clean-Up', 
            'Illegal  dumping', 'Zoning Business', 'Abandoned', 
            'Fire Residential or Commercial', 'Zoning Residential', 
            'How can I request an Alley Clean up?', 'Abandoned Vehicl', 'Other',
            'Vacant Lot Clean-up', 'Right-of-Way', 'Vacant Lot Clean Up', 
            'Vacant Property Complaint', 'Unlicensed Business Complaint', 
            'Fire Department Referrals', 
            'How do I report a commercial building with fire code violations? SERVICE REQUEST',
            'Can I report a property owner for not clearing snow or ice from their sidewalk?',
            'How can I report noise complaints?View Score: 4',
            'How can I report noise complaints?View Score: 9',
            'traffic other', 
            'Who do I call about rats in my neighborhood or home?View Score: 13',
            'How do I report illegal dumping? (SERVICE REQUEST)View Score: 7',
            'Vacant House or Commercial', 'Building Dangerous', 
            'How do I report a Vacant Residential/Commercial property or a Vacant House in violation of the Doors and Windows Ordinance?',
            'Zoning Business', 'Fire Residential or Commercial', 'Abandoned',
            'How do I report a dangerous vacant building?', 
            'Private Property Tree Maintenance or obstructing public access.',
            'How can I request a permit inspection?', 
            'How do I report an illegally parked vehicle?', 
            'How do i report illegal dumping?', 
            'Who do I call about rats in my neighborhood or home?',
            'How do I report a street or sidewalk being blocked?',
            'How do I report a vacant lot? (SERVICE REQUEST)',
            'How do I report a vacant lot?', 'Vacant Lot Clean-up',
            'Where can I report illegal dumping, left on the Street, Sidewalk, or Other Public ROW?',
            'How do I report an overgrown tree on private property affecting adjacent properties?',
            'How can I report air quality complaints?', 
            'How do I report a property owner changing the use of a residential property without proper zoning?',
            'How do I report a dangerous or unsafe garage, fence or retaining wall? (SERVICE REQUEST)',
            'How can I complain about a dumpster on private property or in the public right of way ? (SERVICE REQUEST)',
            'How do I report illegal dumping? (SERVICE REQUEST)',
            'Where can I report illegal dumping, left on the Street, Sidewalk, or Other Public ROW? (SERVICE REQUEST)',
            'How do I report Hazardous Waste or Chemical Spills?',
            'How do I report a commercial property being used for other than what it is legally zoned? (SERVICE REQUEST)',
            'How do I file a complaint for a commercial building with outstanding maintenance issues? (SERVICE REQUEST)',
            'How do I report a commercial property being used for other than what it is legally zoned?', 
            'License and Inspections', 
            'How do I report a dangerous sidewalk? (SERVICE REQUEST)',
            'How can I report noise complaints', 
            'Private Property Tree Maintenance or obstructing public access',
            'PWD Industrial Waste Unit', 'How to enter an LI ESCALATION Request',
            'How do I report a street or sidewalk being blocked? (SERVICE REQUEST)',
            'Right-of-Way', 'How do I complain about advertising signs that are attached to or sitting in front of a building?',
            'Vacant Property Complaint', 'How do I report an overgrown tree on private property affecting adjacent properties',
            'How do I report a dangerous sidewalk', 'Vacant Lot Clean Up', 
            'Unlicensed Business Complaint', 'How do I complain about advertising signs that are attached to or sitting in front of a building? (SERVICE REQUEST)',
            'How do I report a vacant lot? (SERVICE REQUEST)View Score: 3',
            'LI ESCALATION', 'How do I report an abandoned vehicle?View Score: 5',
            'How to report a tree rooted on private property that has fallen and is blocking the sidewalk',
            'How Do I Report A Vacant Lot?', 'How do I report illegal dumping? (SERVICE REQUEST)View Score: 5',
            'How do I report a commercial building with fire code violations? SERVICE REQUEST',
            'How do I report illegal dumping? (SERVICE REQUEST)View Score: 7']

In [ ]:
requests311_2020["subject"].value_counts()

Rubbish/Recyclable Material Collection                                       38199
Illegal Dumping                                                              26304
Maintenance Complaint                                                        24747
Abandoned Automobile                                                         13137
Abandoned Vehicle                                                            11415
                                                                             ...  
How do I report a vacant lot? (SERVICE REQUEST)View Score: 3                     1
How to report a water main break.                                                1
How will the examiner receive a variance?                                        1
How can I report someone smoking at a restaurant?                                1
How do I obtain a permit to hold a Special Event or Festival in the city?        1
Name: subject, Length: 849, dtype: int64

The subject: "'What do I do if I receive a bomb threat against the City of Philadelphia, its employees, elected officials and/or representatives?'," is a little alarming... I don't think the answer is to call 311???


In [ ]:
relevant311 = requests311_2020[requests311_2020['subject'].isin(subjects)]

In [ ]:
relevant311.nunique()

objectid      4179
service_re    4179
subject         57
status           2
status_not       6
service_na       8
service_co       7
agency_res       8
service_no       4
requested_     348
updated_da     352
expected_d     324
closed_dat      82
address       3571
zipcode         45
media_url     1016
lat           3712
lon           3712
geometry      3713
dtype: int64

So many!

In [ ]:
requests311_2020.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 221347 entries, 0 to 621923
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   objectid    221347 non-null  float64 
 1   service_re  221347 non-null  float64 
 2   subject     221347 non-null  object  
 3   status      221347 non-null  object  
 4   status_not  46740 non-null   object  
 5   service_na  221347 non-null  object  
 6   service_co  168193 non-null  object  
 7   agency_res  221347 non-null  object  
 8   service_no  188046 non-null  object  
 9   requested_  221347 non-null  object  
 10  updated_da  221347 non-null  object  
 11  expected_d  178098 non-null  object  
 12  closed_dat  9050 non-null    object  
 13  address     221251 non-null  object  
 14  zipcode     10334 non-null   object  
 15  media_url   31222 non-null   object  
 16  lat         221347 non-null  float64 
 17  lon         221347 non-null  float64 
 18  geometry    2213

In [ ]:
relevant311.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 4179 entries, 25543 to 621840
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   objectid    4179 non-null   float64 
 1   service_re  4179 non-null   float64 
 2   subject     4179 non-null   object  
 3   status      4179 non-null   object  
 4   status_not  1569 non-null   object  
 5   service_na  4179 non-null   object  
 6   service_co  3610 non-null   object  
 7   agency_res  4179 non-null   object  
 8   service_no  2592 non-null   object  
 9   requested_  4179 non-null   object  
 10  updated_da  4179 non-null   object  
 11  expected_d  4178 non-null   object  
 12  closed_dat  1031 non-null   object  
 13  address     4172 non-null   object  
 14  zipcode     566 non-null    object  
 15  media_url   1046 non-null   object  
 16  lat         4179 non-null   float64 
 17  lon         4179 non-null   float64 
 18  geometry    4179 non-null   geomet

In [ ]:
relevant311.tail()

,objectid,service_re,subject,status,status_not,service_na,service_co,agency_res,service_no,requested_,updated_da,expected_d,closed_dat,address,zipcode,media_url,lat,lon,geometry
618306,14493101.0,13220587.0,Vacant House or Commercial,Closed,None,Vacant House or Commercial,SR-LI27,License & Inspections,20 Business Days,2020-03-11,2022-08-22,2020-04-12,2022-08-22,1416 N NEWKIRK ST,19121,None,39.977844,-75.181683,POINT (-75.18168 39.97784)
618321,14493067.0,13215338.0,Zoning Business,Closed,None,Zoning Business,SR-LI28,License & Inspections,20 Business Days,2020-03-09,2022-08-22,2020-04-08,2022-08-22,3510 SPRING GARDEN ST,19104,None,39.962688,-75.193393,POINT (-75.19339 39.96269)
618339,14493396.0,13222918.0,Zoning Business,Closed,None,Zoning Business,SR-LI28,License & Inspections,20 Business Days,2020-03-12,2022-08-22,2020-04-13,2022-08-22,10101-15 BUSTLETON AVE,19116,None,40.102776,-75.028651,POINT (-75.02865 40.10278)
621285,14856701.0,13215686.0,Zoning Business,Open,None,Zoning Business,SR-LI28,License & Inspections,20 Business Days,2020-03-09,2022-10-11,2020-04-08,None,6122-24 BELFIELD AVE,None,None,40.048616,-75.171912,POINT (-75.17191 40.04862)
621840,14950336.0,13214266.0,Fire Residential or Commercial,Open,None,Fire Residential or Commercial,SR-LI16,License & Inspections,20 Business Days,2020-03-09,2022-10-21,2020-04-07,None,201 EAST ST,None,None,40.022404,-75.213932,POINT (-75.21393 40.02240)


In [ ]:
# will L&I escalation show me anything? No not really
requests311_2020[requests311_2020["service_na"].str.contains("LI Escalation")]

,objectid,service_re,subject,status,status_not,service_na,service_co,agency_res,service_no,requested_,updated_da,expected_d,closed_dat,address,zipcode,media_url,lat,lon,geometry
2833,8135397.0,13249015.0,LI Escalation,Closed,None,LI Escalation,None,License & Inspections,2 Business Days,2020-03-25,2020-03-25,2020-03-25,None,243 W TULPEHOCKEN ST,None,None,40.037335,-75.185004,POINT (-75.18500 40.03733)
6205,8135385.0,13248671.0,LI Escalation,Closed,None,LI Escalation,None,License & Inspections,2 Business Days,2020-03-25,2020-03-25,2020-03-25,None,2037 BELLEVUE ST,None,None,40.005279,-75.162739,POINT (-75.16274 40.00528)
8727,8129887.0,13234800.0,LI Escalation,Closed,None,LI Escalation,None,License & Inspections,2 Business Days,2020-03-18,2020-03-24,2020-03-24,None,5642 LEBANON AVE,None,None,39.986616,-75.236120,POINT (-75.23612 39.98662)
8728,8129888.0,13234812.0,LI Escalation,Closed,None,LI Escalation,None,License & Inspections,2 Business Days,2020-03-18,2020-03-24,2020-03-24,None,5642 LEBANON AVE,None,None,39.986616,-75.236120,POINT (-75.23612 39.98662)
24272,8135406.0,13249016.0,LI Escalation,Closed,None,LI Escalation,None,License & Inspections,2 Business Days,2020-03-25,2020-03-25,2020-03-25,None,243 W TULPEHOCKEN ST,None,None,40.037335,-75.185004,POINT (-75.18500 40.03733)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601585,14306076.0,13741968.0,LI Escalation,Closed,None,LI Escalation,None,License & Inspections,2 Business Days,2020-10-27,2022-07-21,None,2020-11-02,4407 COTTMAN AVE,None,None,40.033132,-75.037554,POINT (-75.03755 40.03313)
601644,10956148.0,13690437.0,LI Escalation,Closed,None,LI Escalation,None,License & Inspections,2 Business Days,2020-09-25,2021-09-22,2021-10-14,None,110 N 61ST ST,None,None,39.964087,-75.242372,POINT (-75.24237 39.96409)
602271,11264032.0,13475891.0,LI Escalation,Closed,None,LI Escalation,None,License & Inspections,2 Business Days,2020-07-08,2021-11-16,2021-11-16,None,2350 E YORK ST,None,None,39.978522,-75.125130,POINT (-75.12513 39.97852)
602318,14395158.0,13451165.0,LI Escalation,Closed,None,LI Escalation,None,License & Inspections,2 Business Days,2020-06-25,2022-08-09,None,2020-07-02,220 LOCUST ST # 7HS,None,None,39.945120,-75.145207,POINT (-75.14521 39.94512)


In [ ]:
# why doesn't that return anything from my new dataset?
relevant311[relevant311["subject"].str.contains("LI Escalation")]

,objectid,service_re,subject,status,status_not,service_na,service_co,agency_res,service_no,requested_,updated_da,expected_d,closed_dat,address,zipcode,media_url,lat,lon,geometry


Okay, so L&I Escalation doesn't have much info

In [ ]:
requests311_2020[requests311_2020["service_na"].str.contains("Rubbish/Recyclable Material Collection")].tail()

,objectid,service_re,subject,status,status_not,service_na,service_co,agency_res,service_no,requested_,updated_da,expected_d,closed_dat,address,zipcode,media_url,lat,lon,geometry
620919,11869968.0,13845471.0,Rubbish/Recyclable Material Collection,Closed,None,Rubbish/Recyclable Material Collection,SR-ST03,Streets Department,2 Business Days,2020-12-30,2022-03-12,2021-01-03,None,424 E HORTTER ST,None,None,40.057154,-75.178304,POINT (-75.17830 40.05715)
620920,11869969.0,13844835.0,Rubbish/Recyclable Material Collection,Closed,None,Rubbish/Recyclable Material Collection,SR-ST03,Streets Department,2 Business Days,2020-12-30,2022-03-12,2020-12-31,None,5923 VINE ST,None,None,39.966475,-75.238704,POINT (-75.23870 39.96648)
620921,11869970.0,13844967.0,Rubbish/Recyclable Material Collection,Closed,None,Rubbish/Recyclable Material Collection,SR-ST03,Streets Department,2 Business Days,2020-12-30,2022-03-12,2020-12-31,None,242 N SALFORD ST,None,None,39.965974,-75.239324,POINT (-75.23932 39.96597)
621794,15074549.0,13303982.0,Rubbish/Recyclable Material Collection,Closed,None,Rubbish/Recyclable Material Collection,SR-ST03,Streets Department,2 Business Days,2020-04-20,2022-11-09,2020-04-21,2020-04-22,1226 CHANCELLOR ST,19107,None,39.948623,-75.161742,POINT (-75.16174 39.94862)
621907,14967842.0,13269669.0,Rubbish/Recyclable Material Collection,Closed,None,Rubbish/Recyclable Material Collection,SR-ST03,Streets Department,2 Business Days,2020-04-02,2022-10-26,2020-04-06,2022-09-17,806 FERNON ST,None,None,39.928792,-75.158894,POINT (-75.15889 39.92879)


In [ ]:
requests311_2020[requests311_2020["service_na"].str.contains("Maintenance Complaint")]

,objectid,service_re,subject,status,status_not,service_na,service_co,agency_res,service_no,requested_,updated_da,expected_d,closed_dat,address,zipcode,media_url,lat,lon,geometry
305,12168015.0,13443113.0,Maintenance Complaint,Open,None,Maintenance Complaint,None,License & Inspections,20 Business Days,2020-06-22,2022-03-12,None,None,5548 BOYER ST,19138,None,40.045697,-75.160959,POINT (-75.16096 40.04570)
306,12168017.0,13443440.0,Maintenance Complaint,Open,None,Maintenance Complaint,None,License & Inspections,20 Business Days,2020-06-22,2022-03-12,None,None,1124 S 56TH ST,19143,None,39.943968,-75.233072,POINT (-75.23307 39.94397)
307,12168018.0,13454103.0,Maintenance Complaint,Open,None,Maintenance Complaint,None,License & Inspections,20 Business Days,2020-06-26,2022-03-12,None,None,417 DURFOR ST,19148,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.919445,-75.154625,POINT (-75.15463 39.91944)
308,12168148.0,13815220.0,Maintenance Complaint,Open,None,Maintenance Complaint,None,License & Inspections,20 Business Days,2020-12-11,2022-03-12,None,None,172 MANHEIM ST,19144,None,40.026680,-75.168721,POINT (-75.16872 40.02668)
309,12168149.0,13815434.0,Maintenance Complaint,Open,None,Maintenance Complaint,None,License & Inspections,3 Business Days,2020-12-11,2022-03-12,None,None,111 HERMAN ST,19144,None,40.043246,-75.176913,POINT (-75.17691 40.04325)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621895,14958590.0,13289136.0,Maintenance Complaint,Open,None,Maintenance Complaint,None,License & Inspections,20 Business Days,2020-04-13,2022-10-24,2020-05-12,None,1034 S 51ST ST,None,None,39.943920,-75.220870,POINT (-75.22087 39.94392)
621898,14958591.0,13572826.0,Maintenance Complaint,Open,None,Maintenance Complaint,None,License & Inspections,20 Business Days,2020-07-24,2022-10-24,2020-08-25,None,1034 S 51ST ST,None,None,39.943920,-75.220870,POINT (-75.22087 39.94392)
621899,14964234.0,13659024.0,Maintenance Complaint,Open,None,Maintenance Complaint,None,License & Inspections,20 Business Days,2020-09-09,2022-10-25,2020-10-08,None,1611 E MOYAMENSING AVE,None,None,39.927796,-75.150796,POINT (-75.15080 39.92780)
621912,14977260.0,13757484.0,Maintenance Complaint,Open,None,Maintenance Complaint,None,License & Inspections,3 Business Days,2020-11-04,2022-10-28,2020-11-09,None,138 W LIPPINCOTT ST,None,None,39.997769,-75.130746,POINT (-75.13075 39.99777)


In [ ]:
# how many have links to photos? a lot!
requests311_2020[requests311_2020["media_url"].notna()]

,objectid,service_re,subject,status,status_not,service_na,service_co,agency_res,service_no,requested_,updated_da,expected_d,closed_dat,address,zipcode,media_url,lat,lon,geometry
1,12165628.0,13315073.0,Illegal Dumping,Closed,None,Illegal Dumping,SR-ST02,Streets Department,5 Business Days,2020-04-25,2022-03-12,2020-05-03,None,3100 W THOMPSON ST,None,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.976803,-75.187074,POINT (-75.18707 39.97680)
5,12165648.0,13486115.0,Illegal Dumping,Closed,None,Illegal Dumping,SR-ST02,Streets Department,5 Business Days,2020-07-13,2022-03-12,2020-07-19,None,708 EMILY ST,None,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.923714,-75.158356,POINT (-75.15836 39.92371)
10,12165631.0,13315180.0,Illegal Dumping,Closed,None,Illegal Dumping,SR-ST02,Streets Department,5 Business Days,2020-04-25,2022-03-12,2020-05-03,None,W Norris St,None,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.980218,-75.133689,POINT (-75.13369 39.98022)
17,12165598.0,13346469.0,Illegal Dumping,Closed,None,Illegal Dumping,SR-ST02,Streets Department,5 Business Days,2020-05-10,2022-03-12,2020-05-17,None,5436 BAYNTON ST,None,https://d17aqltn7cihbm.cloudfront.net/uploads/...,40.036482,-75.168687,POINT (-75.16869 40.03648)
21,12165605.0,13312875.0,Illegal Dumping,Closed,None,Illegal Dumping,SR-ST02,Streets Department,5 Business Days,2020-04-23,2022-03-12,2020-04-30,None,4509 WESTMINSTER AVE,None,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.969475,-75.213322,POINT (-75.21332 39.96948)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621878,14973035.0,13451727.0,Maintenance Complaint,Open,None,Maintenance Complaint,None,License & Inspections,20 Business Days,2020-06-25,2022-10-27,2020-07-27,None,112 E ALLEGHENY AVE,None,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.998437,-75.128778,POINT (-75.12878 39.99844)
621883,14965741.0,13357949.0,Traffic (Other),Open,None,Traffic (Other),SR-ST24,Streets Department,60 Business Days,2020-05-14,2022-10-25,2020-08-17,None,EARP ST & S 6TH ST,None,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.931626,-75.154645,POINT (-75.15464 39.93163)
621886,14974282.0,13527176.0,Dangerous Building Complaint,Open,None,Dangerous Building Complaint,None,License & Inspections,15 Business Days,2020-07-17,2022-10-27,2020-08-09,None,1699 MARGARET ST,19124,https://d17aqltn7cihbm.cloudfront.net/uploads/...,40.015354,-75.081031,POINT (-75.08103 40.01535)
621889,14885648.0,13792181.0,Abandoned Automobile,Closed,Information Provided,Abandoned Vehicle,SR-PD01,Police Department,90 Business Days,2020-11-27,2022-10-13,2021-04-19,2022-10-13,122 N PAXON ST,None,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.962039,-75.223708,POINT (-75.22371 39.96204)


In [ ]:
requests311_2020.nunique()

objectid      221347
service_re    221347
subject          849
status             2
status_not        18
service_na        62
service_co        44
agency_res        44
service_no        24
requested_       366
updated_da       700
expected_d       488
closed_dat       293
address       117530
zipcode           50
media_url      29033
lat           126808
lon           126799
geometry      126857
dtype: int64